---
layout: page
title: Inflation and Interest Rates
---

In [1]:
""" import library and package dependencies """
import sys
sys.path.insert(0, '../src')
import os
import datetime
import pytz
from collections import namedtuple
from urllib.parse import quote
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from IPython.display import display, Markdown

from data_loader import load_csv_data, get_last_publish_date
from chart_format import StandardChart, PercentileChart

## [Inflation](inflation.html)

The Consumer Price Index that the Reserve Bank of Australia use to index inflation is published quarterly, one month after the end of each quarter.

See [inflation](inflation.html) for inflation charts, trends and analysis.

## [Interest Rates](interest-rates.html)

The Housing Loan Lending Rates are set by the Reserve Bank of Australia and published monthly within five business days after month end.

See [interest rates](interest-rates.html) for interest rate charts, trends and analysis.

## Inflation and Interest Rates

In [ ]:
""" Load dataframes from other notebooks """
df_inflation = pd.DataFrame()
df_interest = pd.DataFrame()
%store -r df_inflation
%store -r df_interest

In [ ]:
df_interest.tail(10)

,Series ID,FILRSBVRT,FILRSBVOO,FILRHLBVS,FILRHLBVD,FILRHLBVO,FILRHL3YF,FILRHLBVSI,FILRHLBVDI,FILRHLBVDO,...,rolling_1_years_upper,rolling_10_years_lower,rolling_10_years_median,rolling_10_years_upper,rolling_20_years_lower,rolling_20_years_median,rolling_20_years_upper,rolling_30_years_lower,rolling_30_years_median,rolling_30_years_upper
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-30,Jun-2023,8.75,10.51,8.52,7.06,9.04,6.38,9.10,7.50,9.34,...,8.0200,4.8,5.28,5.81,5.28,6.225,7.3375,5.76,6.74,7.79
2023-07-31,Jul-2023,8.75,10.51,8.52,7.06,9.04,6.65,9.10,7.50,9.34,...,8.0825,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2023-08-31,Aug-2023,8.75,10.51,8.52,7.06,9.04,6.58,9.10,7.50,9.34,...,8.3325,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2023-09-30,Sep-2023,8.75,10.51,8.52,7.06,9.04,6.56,9.10,7.50,9.34,...,8.5200,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2023-10-31,Oct-2023,8.75,10.51,8.52,7.06,9.04,6.56,9.10,7.50,9.34,...,8.5200,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2023-11-30,Nov-2023,9.00,10.79,8.77,7.31,9.29,6.80,9.35,7.75,9.59,...,8.5200,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2023-12-31,Dec-2023,9.00,10.79,8.77,7.31,9.29,6.80,9.35,7.75,9.59,...,8.5200,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2024-01-31,Jan-2024,9.00,10.79,8.77,7.31,9.29,6.80,9.35,7.75,9.59,...,8.5825,4.8,5.28,5.81,5.28,6.225,7.3900,5.76,6.74,7.79
2024-02-29,Feb-2024,9.00,10.79,8.77,7.31,9.29,6.80,9.35,7.75,9.59,...,8.7700,4.8,5.28,5.81,5.28,6.225,7.3925,5.76,6.74,7.79


In [ ]:
""" Chart variables """
start_date = pd.to_datetime('1960-01-01')
end_date = pd.to_datetime('2025-01-01') # used to force chart to plot to end of 5 year period
# filter out the dataframe to selected date range
#df_interest = df_interest.loc[df_interest.index >= start_date]
#df_inflation = df_inflation.loc[df_inflation.index >= start_date]
interest_column = 'FILRHLBVS'
inflation_column = 'GCPIAGYP'
# Set 'last_index' to the last date with a valid value (so I can display 'as of <date>' in chart title)
df_interest.columns
#last_index = df_interest[interest_column].last_valid_index()

In [ ]:
def plot_percentiles(plt, df, percentiles, multi_year=0, color_index=0):
    for i, percentile in enumerate(percentiles._fields):
        linestyle = '--' if i != 1 else '-'
        alpha = 0.7 if i != 1 else 1.0
        if i == 0:
            label = f'{str(percentiles[0]) + "th and " + str(percentiles[2]) + "th Percentile"}'
        elif i == 1:
            label = f'{multi_year} Year {percentile.capitalize()}'
        else:
            label = None
        plt.plot(df.index, df[f'rolling_{multi_year}_years_{percentile}'], color=colors[color_index]['color'], linestyle=linestyle, alpha=alpha, label=label)

The Reserve Bank of Australia has often increased base interest rates alledging a high correlation between interest rates and inflation.

This appears a reasonable construct from basic supply and demand economics; when households have high disposable income, demand for non-essential goods and services increases, and if supply is limited, prices increase to meet demand, resulting in inflation. 

Increasing interest rates reduces disposable income (for those with borrowing debt), thus reducing some demand for non-essential goods and services, potentially resulting in some price reductions, and subsequently reducing inflation.

Is there really a high correlation, or is this over simplification? Do lower interest rates subsequently cause inflation to rise, and higher interest rates subsequently cause inflation to reduce?

Perhaps there are other external factors influencing inflation, such as global oil prices, foreign war, pandemic supply chains, that have nothing to do with household disposable income?

""" Prepare simple chart """

chart_params = {
    'chart_title': 'AU Inflation and Housing Loan Lending Rate as at ' + last_index.strftime('%d %b %Y'), 
    'chart_source': 'Source: www.rba.gov.au/statistics/tables', 
    'y_label': 'Rate %', 
    'y_ticks': 2,
    'x_label': 'Date', 
    'x_ticks': 5,
    'start_year': 1960,
    'data_column': inflation_column
}

standard_chart = StandardChart(**chart_params)
plt, colors = standard_chart.base_chart(df_inflation)

plt.plot(df_interest.index, df_interest[interest_column], color=colors[0]['color'], label='Housing Interest Rate')
plt.plot(df_inflation.index, df_inflation[inflation_column], color=colors[1]['color'], label='CPI Inflation Rate')
legend = plt.legend(loc='best')
legend.get_frame().set_alpha(0.98)

plt.show()